# HCTL Model Checking

This notebook illustrates the model-checking capabilities of AEON.py. It allows to model check HCTL properties over (partially defined) Boolean network models.

This notebook assumes that you already have a basic understanding of AEON.py as presented in the remaining notebooks in this folder. However, you should be able to follow this tutorial even if you have not read the remaining notebooks, just keep in mind that you can find more information about the presented features there.



In [1]:
from biodivine_aeon import *
from pathlib import Path

We first show how to utilize the model checking over fully defined Boolean models.

We start by loading a  Boolean network:

In [2]:
bn = BooleanNetwork.from_file("mapk-reduced.aeon")
print(bn)

BooleanNetwork(variables = 17, parameters = 0, regulations = 78)


At this point, all model inputs are represented as variables with constant update functions (this representation tends to have the best compatibility with other tools). This is however inefficient for both BDD representation and for the model-checking algorithm.

Instead, constant input variables can be turned into logical parameters. For larger models, this can save a non-trivial amount of computation time, as the model-checking algorithm can usually deal with these parameters more easily than with the state variables.

Keep in mind that AEON refers to the valuations of such parameters as colors: if the model is using any parameters, the result will be a relation over states and corresponding parameter valuations (colors).

In [3]:
bn = bn.inline_inputs()
print(bn)

BooleanNetwork(variables = 13, parameters = 4, regulations = 60)


We now have a partially specified BN with 13 variables that encode its states, and 4 parameters that encode colors (various concretizations of the network). During model checking, we can analyze which colors (concretizations) satisfy various properties.

#### Extended symbolic transition graph

For the next step, we need to create the `SymbolicAsyncGraph` for the network. This structure actually encodes the network behaviour into a symbolic transition system. We can then use this graph to check for behavioural properties via model checking. 

To be able to perform the HCTL model checking, this structure must additionally also encode the HCTL variables that occur in our formulae. In our case, we will be checking for formulae with at most 3 HCTL variables.

In [4]:
stg = get_extended_stg(bn, 3)

Note that an "extended" symbolic graph is still an instance of `SymbolicAsyncGraph`, hence you can still use it for normal BN analysis. The only difference is that it also admits additional symbolic variables that can be used to represent free variables in HCTL (you can inspect these variables using the `SymbolicContext` object).

In [5]:
print(SymbolicAsyncGraph(bn).symbolic_context().num_extra_state_variables())
print(stg.symbolic_context().num_extra_state_variables())

0
39


#### Running HCTL model checker

The HCTL model checker takes a transition graph and dynamic property encoded as an HCTL formula, and computes which state-color pairs (of the transition graph) satisfy the formula. HCTL enables to express a range of important properties regarding the long-term behaviour of the system.

A simple example of a formula can be `EF (p53)`, where `p53` is a variable of the network. This formula states that "there exists a path to reach a state with active `p53`". The model-checking procedure will return all the state-color pairs for which this is true.

To design more useful (and complex) HCTL formulae, we usually use the following "hybrid" operators (their usage is illustrated later in this notebook):
- A `binder` (`!`) or `exists` (`3`) / `forall` (`V`) are quantifiers that "mark" a state, which can then be referenced later in the formula.
- A `jump` (`@`) allows to jump to the marked state.

For example, formula `!{x}: AX {x}` describes precisely fixed-point states (states with self-loops). It is true for all state-color pairs, such that the state has a self-loop for the given color. The formula can be understood as:
- Bind a current state and name it `x`.
- On all paths, the next state must be again `x`.

Similarly, formula `3{x}: AX {x}` describes existence of fixed-point states. It is true for a color where there exist a fixed-point state.

For more information on the HCTL syntax or the model checker, refer to [this Github page](https://github.com/sybila/biodivine-hctl-model-checker).

#### Checking for general properties regarding attractors

We can use both the basic AEON functionality and HCTL model checking to check for attractors. However, the model checker allows us to check for more detailed properties. It also allows us to specify which attractors we are interested in.

Let's start with the computation of general attractors. We can compare the results of classical attractor computation and the model checker.

In [6]:
# classical computation
attractors = find_attractors(stg)
print(attractors)

# formula for attractors
attractors_mc = model_check("!{x}: AG EF {x}", stg)
print(attractors_mc)

[ColoredVertexSet(cardinality = 3605, unique vertices = 1602, unique colors = 16), ColoredVertexSet(cardinality = 2, unique vertices = 2, unique colors = 2)]
ColoredVertexSet(cardinality = 3607, unique vertices = 1602, unique colors = 16)


The difference is that the model checker always gives us one set of all coloured-states satisfying the formula, whereas the classical AEON computation returns a list of such sets. However, if we combine them, the results are the same.

Let's now focus on some particular types of attractors. We can for instance compute only fixed points, or non-trivial cyclic attractors:

In [7]:
# formula for steady states
fixed_point_attrs = model_check("!{x}: AX {x}", stg)
print(fixed_point_attrs)

ColoredVertexSet(cardinality = 12, unique vertices = 4, unique colors = 10)


In [8]:
# formula for non-trivial cyclic attractors
cyclic_attrs = model_check("AG (!{x}: (AX (~{x} & AF {x})))", stg)
print(cyclic_attrs)

ColoredVertexSet(cardinality = 2, unique vertices = 2, unique colors = 1)


In [9]:
# formula for steady states, in case there are at least two of them
more_than_two_fixed_points = model_check("!{x}: 3{y}: ((@{x}: ~{y} & AX {x}) & (@{y}: AX {y}))", stg)
print(more_than_two_fixed_points)

ColoredVertexSet(cardinality = 4, unique vertices = 4, unique colors = 2)


We can now see that there are 12 fixed-points for 10 different colors. That means there are 6 colors without fixed points.

There is also just one color that admits a cyclic attractor. This attractor comprises two states.

And finally, there are two colors that admit more than one fixed point. In our case, both of them admit two fixed-points.

Let's imagine we are now interested only in fixed-point attractors with `p53` active.

In [10]:
fixed_point_p53 = model_check("!{x}: AX ({x} & v_p53)", stg)
print(fixed_point_p53)

attractors_non_p53 = fixed_point_attrs.minus(fixed_point_p53)
attractors_non_p53_v2 = model_check("!{x}: AX ({x} & ~v_p53)", stg)
print(attractors_non_p53)

assert attractors_non_p53 == attractors_non_p53_v2

ColoredVertexSet(cardinality = 10, unique vertices = 2, unique colors = 9)
ColoredVertexSet(cardinality = 2, unique vertices = 2, unique colors = 1)


We see that for 9 out of 10 colors (that admit fixed points), the p53 is stably activated. However, for the one remaining color, the p53 is deactivated in both fixed points.

#### Generating and exploring syntactic tree of HCTL formulae

It is also possible to build and explore the syntactic trees for any HCTL formulae. The tree might be more suitable representation than a string.

In [11]:
tree = HctlTreeNode("!{x}: AG EF {x}", bn)
print(tree)

(Bind {x}: (Ag (Ef {x})))


The parsing process also supports some renaming techniques or modifications in order to optimize model-checking computation. If you'd prefer to parse the tree exactly as the formula is written (and exclude these modifications), there is another method for that. However, note that trees that are created this way should not be directly passed to the model-checking procedure as they can contain unsupported combinations of operators and variables.

We can also analyse the syntactic tree in different ways, such as compute the set of all HCTL variables in the formula.

In [12]:
tree = HctlTreeNode("!{x}: 3{y}: ((@{x}: ~{y} & AX {x}) & (@{y}: AX {y}))", bn)
print(tree)

tree_exact = HctlTreeNode.build_exact_from_formula("!{x}: 3{y}: ((@{x}: ~{y} & AX {x}) & (@{y}: AX {y}))")
print(tree_exact)

(Bind {x}: (Exists {xx}: ((Jump {x}: ((~ {xx}) & (Ax {x}))) & (Jump {xx}: (Ax {xx})))))
(Bind {x}: (Exists {y}: ((Jump {x}: ((~ {y}) & (Ax {x}))) & (Jump {y}: (Ax {y})))))


In [13]:
print(tree.collect_unique_hctl_vars())
print(tree_exact.collect_unique_hctl_vars())

{'xx', 'x'}
{'x', 'y'}


Trees can also be traversed and analyzed node by node.

In [14]:
node = HctlTreeNode("!{x}: AX ({x} & v_p53)", bn)
print(node)

child = node.get_children()[0]
print(child)

child_operator = child.get_operator()
print(child_operator)

(Bind {x}: (Ax ({x} & v_p53)))
(Ax ({x} & v_p53))
Ax


Model checking can be run directly on the trees (the same method, but instead of supplying formula as a string, a tree is given).

In [15]:
tree = HctlTreeNode("!{x}: AG EF {x}", bn)
result_v1 = model_check(tree, stg)

result_v2 = model_check("!{x}: AG EF {x}", stg)
assert result_v1 == result_v2

#### Using wild-card propositions during model checking

Sometimes we might want to pre-compute certain sets of states and later re-use them during further computation. This is possible, as we can construct `extended` formulae with special `wild-card propositions` that are evaluated as provided (colored) sets.

Let's first evaluate some arbitrary sub-formula. Note that in order to re-use the result of the model-checking procedure in subsequent computation, we must turn of the so-called `sanitization` step. This step (which is active by default) cleans the underlaying BDDs by removing some potentially redundant symbolic variables used during model checking (see above in the comparison between "normal" and "extended" STG). However, these variables are still needed in our case.

In [16]:
intermediate_result = model_check("!{x}: AX ({x} & v_p53)", stg, sanitize=False)

Then, we'll define an extended formula with a special proposition that we will use to reference the previous result. These special propositions are given as `%proposition-name%`. To evaluate the extended formula, we use a special function, where we provide a context for each wild-card proposition.

In [17]:
extended_formula = "EF %p%"
context_mapping = {"p": intermediate_result}

result = model_check_extended(extended_formula, stg, context_mapping)

We can check that the result is the same as if we computed everything in one step.

In [18]:
result_one_step = model_check("EF (!{x}: AX ({x} & v_p53))", stg)
assert result == result_one_step